# Working in AE5 with Multiple Envs
## Basics
- Anaconda enterprise launches a pod with 4 containers:
  - two with the editor image, one for the actual editor the other for synching repo
  - two with proxy image, one for each above
- the ae-editor image, is huge in docker terms, it is the current mechnisim we adopted to deliver the pre-canned environments to all the cluster node (this will be handled by the container runtime and will be cached on all hosts, if configured with high performence storage, that will provide better perormence.
- the canned environments are:

In [9]:
!conda env list | awk '!/#/ {print $1}'

base
anaconda201908_py37
anaconda44_py27
anaconda44_py35
anaconda44_py36
anaconda44_r
anaconda50_hadoop
anaconda50_impyla
anaconda50_py27
anaconda50_py35
anaconda50_py36
anaconda50_sas
lab_launch
py36
sync_launch



base                  
anaconda201908_py37     
anaconda44_py27          
anaconda44_py35         
anaconda44_py36          
anaconda44_r             
anaconda50_hadoop        
anaconda50_impyla       
anaconda50_py27          
anaconda50_py35         
anaconda50_py36         
anaconda50_sas          
lab_launch             
py36                   
sync_launch    

- base is where conda is installed
- lab_launch is where the editor environment is installed (Jupyter notebook/lab zeppelin, etc...) as well as all the anaconda-project and other supporting tools for preparing the session.
- base is in base /opt/continuum/anaconda 
- all other envs are /opt/continuum/anaconda/envs/<env-name>
- lab_launch has many binaries linked from base (like conda)

ANACONDA_PROJECT_ENVS_PATH is set so that new environment created by anaconda-project are created there, and not where the command is executed

In [2]:
!env | grep ANACONDA_PROJECT_ENVS_PATH

ANACONDA_PROJECT_ENVS_PATH=/opt/continuum/anaconda/envs


In [ ]:
!conda env list